## TensorFlow キャッチアップ

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.datasets import mnist

In [16]:
# constant

t1 = time.time() 

# 計算開始

a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)

print(add)                               # エッジの説明が格納されているだけ

with tf.Session() as sess:
    output = sess.run(add)     # Session.run()にエッジを格納すると計算がなされる
    print(output) # 12
sess.close()

# 計算終了


t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

Tensor("Add_13:0", shape=(), dtype=int32)
12
経過時間：0.0179288387298584


In [17]:
# 処理前の時刻
t1 = time.time() 

# 計算開始

a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) # 12

# 計算終了


t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

# 簡単な計算だとNumpyのが早い・記述も楽

12
経過時間：0.0005831718444824219


In [18]:
# placeholder

t1 = time.time() 

# 計算開始

c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)

with tf.Session() as sess:
    output = sess.run(add, feed_dict={c:5, d:7})
    print(output) # 12
sess.close()
# 計算終了

t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

12
経過時間：0.017274141311645508


- ロジスティック回帰

In [0]:
# 入力が1, 1の場合にのみ1と出力したい
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [0]:
# Valiable → 学習により更新を行う値であることを宣言(重み・バイアスなど)

W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [0]:
# matmul → np.dotに近い挙動
y = tf.sigmoid(tf.matmul(x, W) + b)

# クロスエントロピー誤差
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

# 二乗和誤差
sse = tf.reduce_sum(tf.square(y - t))

# 勾配降下法(0.1は学習率)
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

# 予測・正確度
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 上記のエッジはrunできんかった

#with tf.Session() as sess:
    #output1 = sess.run(y)
    #output2 = sess.run(cross_entropy)
    #output3 = sess.run(sse)
    #output4 = sess.run(train_step)
    #output5 = sess.run(correct_prediction)
    #output6 = sess.run(accuracy)
#sess.close()

In [22]:
sess = tf.Session()

# 初期化
sess.run(tf.global_variables_initializer())

# 学習
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
    # 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print (f'epoch: {epoch}, Accuracy: {acc_val}')
        
        # h_thetaの確認
        mat = tf.matmul(x, W)
        y = tf.sigmoid(mat + b)
        
        mat = sess.run(mat, feed_dict={
            x:x_train,
            t:y_train
        })
        
        print(f'mat: {mat} mat.shape: {mat.shape}')

epoch: 0, Accuracy: 0.75
mat: [[0.]
 [0.]
 [0.]
 [0.]] mat.shape: (4, 1)
epoch: 100, Accuracy: 1.0
mat: [[0.       ]
 [1.7671356]
 [1.7671356]
 [3.5342712]] mat.shape: (4, 1)
epoch: 200, Accuracy: 1.0
mat: [[0.      ]
 [2.702048]
 [2.702048]
 [5.404096]] mat.shape: (4, 1)
epoch: 300, Accuracy: 1.0
mat: [[0.      ]
 [3.345771]
 [3.345771]
 [6.691542]] mat.shape: (4, 1)
epoch: 400, Accuracy: 1.0
mat: [[0.       ]
 [3.8412285]
 [3.8412285]
 [7.682457 ]] mat.shape: (4, 1)
epoch: 500, Accuracy: 1.0
mat: [[0.      ]
 [4.244355]
 [4.244355]
 [8.48871 ]] mat.shape: (4, 1)
epoch: 600, Accuracy: 1.0
mat: [[0.       ]
 [4.5839767]
 [4.5839767]
 [9.1679535]] mat.shape: (4, 1)
epoch: 700, Accuracy: 1.0
mat: [[0.       ]
 [4.8771544]
 [4.8771544]
 [9.754309 ]] mat.shape: (4, 1)
epoch: 800, Accuracy: 1.0
mat: [[ 0.       ]
 [ 5.1348853]
 [ 5.1348853]
 [10.269771 ]] mat.shape: (4, 1)
epoch: 900, Accuracy: 1.0
mat: [[ 0.     ]
 [ 5.36469]
 [ 5.36469]
 [10.72938]] mat.shape: (4, 1)


In [23]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})

#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})

print(classified)

print(prob)

sess.close()

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9648671e-04]
 [4.9049795e-02]
 [4.9049795e-02]
 [9.3120372e-01]]


## 【問題1】スクラッチを振り返る  
どのような実装をしたかを列挙する
- 重みの初期化
- エポックのループ
- パラメータの更新
- 誤差の計算・保持
- 予測(predict・predict_proba)


## 【問題2】スクラッチとTensorFlowの対応を考える

In [0]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [0]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [26]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 37.0548, val_loss : 31.6318, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 8.3026, val_loss : 24.7911, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 5.4770, val_loss : 15.2191, acc : 0.750, val_acc : 0.688
Epoch 3, loss : 0.1323, val_loss : 7.7516, acc : 1.000, val_acc : 0.500
Epoch 4, loss : 3.2350, val_loss : 12.1814, acc : 0.750, val_acc : 0.688
Epoch 5, loss : 0.0874, val_loss : 7.7894, acc : 1.000, val_acc : 0.500
Epoch 6, loss : 0.0000, val_loss : 5.0974, acc : 1.000, val_acc : 0.750
Epoch 7, loss : 0.0000, val_loss : 3.9478, acc : 1.000, val_acc : 0.750
Epoch 8, loss : 0.0000, val_loss : 3.2221, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 1.5515, acc : 1.000, val_acc : 0.875
test_acc : 0.800


## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [27]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ワンホットライブラリのインスタンス作成
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(tf.nn.softmax(logits),1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, loss : 1.8643, val_loss : 1.9158, acc : 0.833, val_acc : 0.833
Epoch 1, loss : 2.7703, val_loss : 2.6188, acc : 0.667, val_acc : 0.833
Epoch 2, loss : 1.2170, val_loss : 3.6050, acc : 0.833, val_acc : 0.792
Epoch 3, loss : 0.0446, val_loss : 2.3526, acc : 1.000, val_acc : 0.875
Epoch 4, loss : 0.0000, val_loss : 0.4288, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0000, val_loss : 1.5085, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 0.3945, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 0.8306, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0000, val_loss : 1.3256, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.7531, val_loss : 6.3527, acc : 0.667, val_acc : 0.708
test_acc : 0.667


## 【問題4】House Pricesのモデルを作成

In [28]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
y = y[:, np.newaxis]

scaler = StandardScaler()
scaler.fit(y)
y = scaler.transform(y)

X = np.array(X)


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 1e-5
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, 1])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.square(Y - logits))
# 最適化手法
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))

Epoch 0, loss : 504.4492, val_loss : 1194.7134
Epoch 1, loss : 483.1086, val_loss : 1141.0264
Epoch 2, loss : 462.2194, val_loss : 1089.1327
Epoch 3, loss : 441.9364, val_loss : 1039.2843
Epoch 4, loss : 422.2136, val_loss : 991.6172
Epoch 5, loss : 403.2039, val_loss : 946.0821
Epoch 6, loss : 384.9580, val_loss : 902.5712
Epoch 7, loss : 367.4388, val_loss : 861.0481
Epoch 8, loss : 350.6330, val_loss : 821.4575
Epoch 9, loss : 334.5052, val_loss : 783.7001
Epoch 10, loss : 319.0035, val_loss : 747.6428
Epoch 11, loss : 304.1477, val_loss : 713.2308
Epoch 12, loss : 289.8966, val_loss : 680.3714
Epoch 13, loss : 276.2477, val_loss : 649.0452
Epoch 14, loss : 263.2149, val_loss : 619.2173
Epoch 15, loss : 250.6988, val_loss : 590.8048
Epoch 16, loss : 238.6835, val_loss : 563.7975
Epoch 17, loss : 227.1550, val_loss : 538.1102
Epoch 18, loss : 216.0928, val_loss : 513.6505
Epoch 19, loss : 205.4890, val_loss : 490.3732
Epoch 20, loss : 195.3773, val_loss : 468.2406
Epoch 21, loss : 18

## 【問題5】MNISTのモデルを作成

In [33]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 平坦化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# ワンホットライブラリのインスタンス作成
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])


# 学習データをスプリット
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(tf.nn.softmax(logits),1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 1.3353, val_loss : 1.2701, acc : 0.600, val_acc : 0.664
Epoch 1, loss : 0.8308, val_loss : 2.0287, acc : 0.700, val_acc : 0.550
Epoch 2, loss : 0.7625, val_loss : 0.9362, acc : 0.700, val_acc : 0.707
Epoch 3, loss : 0.2680, val_loss : 0.6330, acc : 0.900, val_acc : 0.866
Epoch 4, loss : 0.5722, val_loss : 0.4824, acc : 0.800, val_acc : 0.887
Epoch 5, loss : 0.4279, val_loss : 0.4697, acc : 0.900, val_acc : 0.895
Epoch 6, loss : 0.4857, val_loss : 0.4011, acc : 0.900, val_acc : 0.910
Epoch 7, loss : 0.7445, val_loss : 0.3637, acc : 0.900, val_acc : 0.915
Epoch 8, loss : 0.4065, val_loss : 0.3369, acc : 0.900, val_acc : 0.921
Epoch 9, loss : 0.2142, val_loss : 0.3636, acc : 0.900, val_acc : 0.920
test_acc : 0.923


In [0]:
tf.summary.FileWriter('test', sess.graph)
!tensorboard --logdir=test --port=8010

TensorBoard 1.13.1 at http://d0e87c770357:8010 (Press CTRL+C to quit)


http://localhost:8010